In [1]:
import pandas as pd

##### Load data

In [2]:
well_map = pd.read_csv(r'../../raw_data/well_PTID_map.csv')
rid_map = pd.read_csv(r'../../raw_data/RID_PTID_map.csv')

well_dict = {well_map.iloc[i]["WGS_SAMPLE_NUMBER"]: well_map.iloc[i]["ADNI_PTID"] for i in range(well_map.shape[0])}
rid_dict =  {rid_map.iloc[i]["RID"]:                rid_map.iloc[i]["PTID"]       for i in range(rid_map.shape[0])}

In [3]:
cnv = pd.read_csv(r'../../raw_data/cnv.tsv', sep='\t')
expr = pd.read_csv(r'../../raw_data/gene_expression.tsv', sep='\t')
metab = pd.read_csv(r'../../raw_data/metabolomics.csv')
metab_dict = pd.read_csv(r'../../raw_data/metabolomics_dict.csv')

##### Create new dataframes with PTIDs only

In [9]:
expr_dict = {}
for i in range(2, len(expr.columns)):
    expr_dict[expr.columns[i]] = expr.columns[i][1:]

expr_new = expr.rename(columns=expr_dict)

In [20]:
metab_rids = metab["RID"].to_numpy()
metab_ptids = []
for i in range(metab_rids.shape[0]):
    if metab_rids[i] in rid_dict.keys():
        metab_ptids.append(rid_dict[metab_rids[i]])

metab_new = metab.merge(rid_map[["RID"]], how='inner', on="RID").drop(columns=["RID"])
metab_new["PTID"] = metab_ptids

cols = metab_new.columns.tolist()
cols.remove("PTID")
cols.insert(1, "PTID")
metab_new = metab_new[cols]

In [13]:
cnv_new = cnv.rename(columns=well_dict)

##### Transpose data and compress feature names

In [29]:
expr_new['Symbol']



0            SAMD7
1           ARRDC5
2        HIST1H2AJ
3           C2CD4B
4            RAB3D
           ...    
17844       ZYG11A
17845       ZYG11B
17846          ZYX
17847        ZZEF1
17848         ZZZ3
Name: Symbol, Length: 17849, dtype: object

In [22]:
expr_new.to_csv(r'../../data/gene_expression.csv')
metab_new.to_csv(r'../../data/metabolomics.csv')
cnv_new.to_csv(r'../../data/cnv.csv')